In [49]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
# Load and view the dataset
df = pd.read_csv('../data/raw/insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


There are 7 columns and 1338 rows. The target column is charges, and we have a mix of numeric and non-numeric columns to predict it.

We do not have any IDs or datetime data.

## Basic Data Check: 
1. Missing
2. Duplicates
3. Data types
4. Outliers 
5. Categorical encoding
6. Feature scaling

### 1. Missing Data

In [51]:
# Check the status of missing data
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

At a surface level it looks like there is no missing data, but there could be a filler value for missing data.

In [71]:
# Are there any sentinel #'s like -1 or 999 as a placeholder for "missing value"?
print('Numeric column distribution at a glance:')
print('=========================================')
df.describe()

Numeric column distribution at a glance:


,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


With the numeric columns there are no values like -1 for the min value or 999 in the max would be placeholders for missing data.

In [72]:
non_num_cols = df.select_dtypes(exclude="number").columns.to_list()

print('Non-numeric column unique values:')
print('=================================')
for i in non_num_cols:
    print(df[i].value_counts(),"\n")

Non-numeric column unique values:
sex
male      676
female    662
Name: count, dtype: int64 

smoker
no     1064
yes     274
Name: count, dtype: int64 

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64 



For the non-numeric columns, I do not see any potential missing data indicator as a category.

There is no noticeable missing data.

### 2. Duplicates

In [73]:
num_dup = df.duplicated().sum()
print(f"The number of duplicates: {num_dup}\n")
dup_rows = df[df.duplicated(keep=False)]
print(dup_rows)

The number of duplicates: 1

     age   sex    bmi  children smoker     region    charges
195   19  male  30.59         0     no  northwest  1639.5631
581   19  male  30.59         0     no  northwest  1639.5631


One exact duplicate exists out of the 1338 rows. If there were more examples, the row count frequency could have meaning such as logs, transactions, or time series which could be aggregated. However, since there is only one, I will choose to remove it. 

In [74]:
df_unique = df.drop_duplicates()
print(f"The value counts of duplicates:\n {df_unique.duplicated().value_counts()}")
print(f"The shape: {df_unique.shape}")

The value counts of duplicates:
 False    1337
Name: count, dtype: int64
The shape: (1337, 7)


In [75]:
# Reset the index
df = df_unique.reset_index(drop=True)

### 3. Data Types

Right here we need to be aware of semantic data type. Meaning we want to match the variable data type for what it truly means in the real world.

In [85]:
print(df.select_dtypes(include='number').dtypes)
df.select_dtypes(include='number')

age           int64
bmi         float64
children      int64
charges     float64
dtype: object


,age,bmi,children,charges
0,19,27.900,0,16884.92400
1,18,33.770,1,1725.55230
2,28,33.000,3,4449.46200
3,33,22.705,0,21984.47061
4,32,28.880,0,3866.85520
...,...,...,...,...
1332,50,30.970,3,10600.54830
1333,18,31.920,0,2205.98080
1334,18,36.850,0,1629.83350
1335,21,25.800,0,2007.94500


In [89]:
df.describe()

,age,bmi,children,charges
count,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,30.663452,1.095737,13279.121487
std,14.044333,6.100468,1.205571,12110.359656
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.290000,0.000000,4746.344000
50%,39.000000,30.400000,1.000000,9386.161300
75%,51.000000,34.700000,2.000000,16657.717450
max,64.000000,53.130000,5.000000,63770.428010


age, bmi, children, and charges have appropriate datatypes that are representative.

In [88]:
print(df.select_dtypes(exclude='number').dtypes)
df.select_dtypes(exclude='number')

sex       object
smoker    object
region    object
dtype: object


,sex,smoker,region
0,female,yes,southwest
1,male,no,southeast
2,male,no,southeast
3,male,no,northwest
4,male,no,northwest
...,...,...,...
1332,male,no,northwest
1333,female,no,northeast
1334,female,no,southeast
1335,female,no,southwest


In [90]:
non_num_cols = df.select_dtypes(exclude="number").columns.to_list()
for i in non_num_cols:
    print(df[i].value_counts(),"\n")

sex
male      675
female    662
Name: count, dtype: int64 

smoker
no     1063
yes     274
Name: count, dtype: int64 

region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64 



There are no numeric columns disguised as an object column like "1" or "NA". There are no mixed datatypes either.

However, later these columns should be encoded to be numerical. Given the logic, there they are nominal, so one-hot encoding would work.

Moreover, the cardinality is not high and there are no rare categories to aggregate.

### 4. Outliers